In [1]:
from quantumsim import *
from SurfaceCode import *

<h1> Four Cycle Block, surface code using QuantumSim </h1>
<img src="./pictures/four_cycle_surface_code_white_background.png" alt="Bit flip correction circuit" style="width:50%;"/>

In [2]:
circuit = Circuit(4)

# Encode state among two physical qubits
circuit.hadamard(0)
circuit.cnot(0, 1)

# Prepare anicallary qubits 
circuit.hadamard(2)
circuit.hadamard(3)

# Introduce bit flip on one data qubit
circuit.bitflip_error(0)

# Introduce phase flip on one data qubit
circuit.phaseflip_error(0)

# Syndrome extraction 
circuit.cnot(2, 0)
circuit.cnot(2, 1)

# Syndrome extraction - Check for bit flips
circuit.controlled_pauli_z(3, 0)
circuit.controlled_pauli_z(3, 1)

# Prepare for measurement 
circuit.hadamard(2)
circuit.hadamard(3)

circuit.execute()
circuit.print_gates_and_descriptions()
circuit.measure(print_state=True)
print(circuit.get_classical_state_of_qubit_as_string(2))
print(circuit.get_classical_state_of_qubit_as_string(3))



H...	Hadamard on qubit 0
*X..	CNOT with control qubit 0 and target qubit 1
..H.	Hadamard on qubit 2
...H	Hadamard on qubit 3
X...	Bit-flip error (Pauli X) on qubit 0
Z...	Phase-flip error (Pauli Z) on qubit 0
X.*.	CNOT with control qubit 2 and target qubit 0
.X*.	CNOT with control qubit 2 and target qubit 1
Z..*	Controlled Pauli Z with control qubit 3 and target qubit 0
.Z.*	Controlled Pauli Z with control qubit 3 and target qubit 1
..H.	Hadamard on qubit 2
...H	Hadamard on qubit 3
Measured state:
|0111>
|1>	 Measured value of qubit 2
|1>	 Measured value of qubit 3


<h1> Four Cycle Block, surface code using Qiskit </h1>

In [3]:
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram

# Create a new circuit with two qubits
qc = QuantumCircuit(4, 2)

qc.h(0)
qc.cx(0, 1)

qc.z(0)
# qc.x(0)

qc.h(2)
qc.h(3)


qc.cx(2, 0)
qc.cx(2, 1)

qc.cz(3, 0)
qc.cz(3, 1)

qc.h(2)
qc.h(3)

qc.measure(2, 0)
qc.measure(3, 1)


# Return a drawing of the circuit using MatPlotLib ("mpl"). This is the
# last line of the cell, so the drawing appears in the cell output.
# Remove the "mpl" argument to get a text drawing.
qc.draw("mpl")

# backend = BasicSimulator()
# result = backend.run(qc, shots=100).result()   
# counts = result.get_counts()                    
# plot_histogram(counts)




ModuleNotFoundError: No module named 'qiskit'

<h1> The [[5, 1, 2]] Detection code </h1>
<img src="./pictures/5_1_2_surface_code_white_background.png" alt="Bit flip correction circuit" style="width:50%;"/>

In [4]:
surfaceCode = SurfaceCode(3)



Surface code initliazed with lattice: 5
Total qubits used for surface: 25
Qubit: D1 type: Data	x: 0 y: 1
Qubit: A1 type: Ancillary	x: 1 y: 1
Qubit: D2 type: Data	x: 2 y: 1
Qubit: A2 type: Ancillary	x: 3 y: 1
Qubit: D3 type: Data	x: 4 y: 1
Qubit: A3 type: Ancillary	x: 0 y: 2
Qubit: D4 type: Data	x: 1 y: 2
Qubit: A4 type: Ancillary	x: 2 y: 2
Qubit: D5 type: Data	x: 3 y: 2
Qubit: A5 type: Ancillary	x: 4 y: 2
Qubit: D6 type: Data	x: 0 y: 3
Qubit: A6 type: Ancillary	x: 1 y: 3
Qubit: D7 type: Data	x: 2 y: 3
Qubit: A7 type: Ancillary	x: 3 y: 3
Qubit: D8 type: Data	x: 4 y: 3
Qubit: A8 type: Ancillary	x: 0 y: 4
Qubit: D9 type: Data	x: 1 y: 4
Qubit: A9 type: Ancillary	x: 2 y: 4
Qubit: D10 type: Data	x: 3 y: 4
Qubit: A10 type: Ancillary	x: 4 y: 4
Qubit: D11 type: Data	x: 0 y: 5
Qubit: A11 type: Ancillary	x: 1 y: 5
Qubit: D12 type: Data	x: 2 y: 5
Qubit: A12 type: Ancillary	x: 3 y: 5
Qubit: D13 type: Data	x: 4 y: 5


In [ ]:
import quantumsim as sim
import scipy.sparse as sps
import numpy as np

N = 3
index = 2
# Changed matrix froim DOK matrix to CSR matrix for optimization
sparse_matrix  = sps.dok_matrix((2**N, 1), dtype=complex)
sparse_matrix [index] = 1

# Convert to dense Numpy array
dense_array = sparse_matrix.toarray()
print(dense_array)

# Flatten the dense array to a state vector
state_vector = dense_array.flatten()
print(state_vector)

# Calculate the outer product to get the density matrix of the state vector
density_matrix = np.outer(state_vector, state_vector.conj().T)
print(density_matrix)

# Covert density matrix to probabilities matrix
probabilities_matrix = np.diag(density_matrix).real
print(probabilities_matrix)

measured_index = np.random.choice(len(probabilities_matrix), p=probabilities_matrix)
print(measured_index)

[[0.+0.j]
 [0.+0.j]
 [1.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]
[0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
[0. 0. 1. 0. 0. 0. 0. 0.]
2
